# Simple House Prices Prediction (Linear Regression)

## Background

House prices is very important aspect to many stakeholders, namely developers, investors, homeowners, and many more (Limsombunchai et al., 2004). Predicting a house prices could improve the efficiency of the real estate market (Limsombunchai et al., 2004). Also, house prices prediction could fill up an important information gap by considering multiple variables that could predict house prices (Limsombunchai et al., 2004).

## Scope

This notebook restricted to:
1. Boston House Prices Dataset (source: https://www.kaggle.com/vikrishnan/boston-house-prices)
2. Using Linear Regression Model

## Purpose

This notebook purpose is creating model by using simple linear regression.

## Data Analysis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Read Data

In [ ]:
header_names=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 
              'DIS', 'RAD', 'TAX', 'PTRATION', 'B', 'LSTAT', 'PRICE']
df = pd.read_csv('../input/boston-house-prices/housing.csv', names=header_names, delim_whitespace=True)
df.head()

### Exploratory Data Analysis

In [ ]:
df.describe()

In [ ]:
df.info()

### Checking Missing Values

In [ ]:
missing_values = df.isnull().sum()
missing_values

### Checking Duplicate Rows

In [ ]:
total_duplicated = df.duplicated().sum()
total_duplicated

### Checking Outliers

In [ ]:
'''
Check number of outliers
'''
def check_outlier(data_frame):
    Q1 = data_frame.quantile(0.25) #First Quartile
    Q3 = data_frame.quantile(0.75) #Second Quartile
    IQR = Q3 - Q1 #Interquartile
    
    return (data_frame < (Q1 - 1.5 * IQR)) | (data_frame > (Q3 + 1.5 * IQR))
    
outlier_df = check_outlier(df)
print(outlier_df.sum())

In [ ]:
'''
Visualize outliers
'''
def visualize_outliers(data_frame):
    for column_name in data_frame.columns:
        sns.boxplot(x=df[column_name])
        plt.show()

visualize_outliers(df)

In [ ]:
'''
Delete data that considered as outlier

Reference: https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
'''

df_no_out = df[~outlier_df.any(axis=1)]
df_no_out

### Separate Feature and Target Label

In [ ]:
X = df_no_out.iloc[:, :-1] #Features
y = df_no_out.iloc[:, -1] #Target

In [ ]:
X.head(3)

In [ ]:
y.head(3)

### Split Train and Test Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print('X Train Shape: ', X_train.shape)
print('X Test Shape: ', X_test.shape)
print('Y Train Shape: ', y_train.shape)
print('Y Test Shape: ', y_test.shape)

### Train Model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

model_pipeline = Pipeline([
  ('scale', MinMaxScaler()),
  ('regression', LinearRegression())
])

model_pipeline.fit(X_train, y_train)

In [ ]:
predictions = model_pipeline.predict(X_test)
print(predictions)

### Comparison between Actual and Predicted Data

In [ ]:
comparison_df = pd.DataFrame({'actual': y_test, 'predicted': predictions})
comparison_df.head(5)

### Model Evaluation

In [ ]:
'''
Evaluation using several metrics
'''
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

metrics_mse = mean_squared_error(y_test, predictions)
metrics_rmse = np.sqrt(metrics_mse)
metrics_mae = mean_absolute_error(y_test, predictions)
metrics_r2 = r2_score(y_test, predictions)

evaluation_indicators = ['MSE (Mean Square Error)', 'RMSE (Root Mean Square Error)', 
                        'MAE (Mean Absolute Error)', 'R2 (Coefficient of Determination)']

evaluation_values = [metrics_mse, metrics_rmse, metrics_mae, metrics_r2]

model_evaluation_df = pd.DataFrame({'indicator': evaluation_indicators, 'value': evaluation_values})

model_evaluation_df

## References

Limsombunchai, Visit & Gan, Christopher & Lee, Minsoo. (2004). House Price Prediction: Hedonic Price Model vs. Artificial Neural Network. American Journal of Applied Sciences. 1. 10.3844/ajassp.2004.193.201. 